In [1]:
import pickle
from pathlib import Path

from enbios import Experiment

In [2]:
# Open a pickled experiment object. If it does not exist, it will be created
BRIGHTWAY_ECOINVENT_391_PROJECT_NAME = "ecoinvent_391"
existing_experiment = Path("exp.pickle")
if existing_experiment.exists():
    experiment = pickle.load(existing_experiment.open("rb"))
else:
    experiment = Experiment(
        {
            "adapters": [
                {
                    "adapter_name": "brightway-adapter",
                    "config": {"bw_project": BRIGHTWAY_ECOINVENT_391_PROJECT_NAME},
                    "methods": {
                        "GWP1000": (
                            "ReCiPe 2016 v1.03, midpoint (H)",
                            "climate change",
                            "global warming potential (GWP1000)",
                        )
                    },
                }
            ],
            "hierarchy": {
                "name": "root",
                "aggregator": "sum",
                "children": [
                    {
                        "name": "wind",
                        "aggregator": "sum",
                        "children": [
                            {
                                "name": "electricity production, wind, 1-3MW turbine, onshore",
                                "adapter": "bw",
                                "config": {
                                    "code": "ed3da88fc23311ee183e9ffd376de89b",
                                    "default_output": {
                                        "unit": "kilowatt_hour",
                                        "magnitude": 3,
                                    },
                                },
                            },
                            {
                                "name": "electricity production, wind, 1-3MW turbine, offshore",
                                "adapter": "bw",
                                "config": {"code": "6ebfe52dc3ef5b4d35bb603b03559023"},
                            },
                        ],
                    },
                    {
                        "name": "solar",
                        "aggregator": "sum",
                        "children": [
                            {
                                "name": "electricity production, solar tower power plant, 20 MW",
                                "adapter": "bw",
                                "config": {"code": "f2700b2ffcb6b32143a6f95d9cca1721"},
                            },
                            {
                                "name": "electricity production, solar thermal parabolic trough, 50 MW",
                                "adapter": "bw",
                                "config": {"code": "19040cdacdbf038e2f6ad59814f7a9ed"},
                            },
                        ],
                    },
                ],
            },
        }
    )
    experiment.run()

In [10]:
experiment.get_simplified_hierarchy(print_it=True);

In [4]:
# let's try a flat hierarchy with all nodes directly under the root. These are all structural (bottom) nodes of our hierarchy
experiment.structural_nodes_names

In [5]:
# let's build the flat hierarchy
alt_hierarchy = {
    "name": "root",
    "aggregator": "sum",
    "children": experiment.structural_nodes_names,
}

alt_hierarchy

__Note how, we can simply pass the names of structural nodes in a list as children.__ That is because when calculating the results for an alternative hierarchy, we do not change the outputs and thus the results of the structural nodes - __Only aggregation of outputs and results is reevaluated!__This also allows new and different aggregators and structural nodes. 

In [9]:
import json

# we can pass the new hierarchy to result_to_dict, for the whole experiment (all scenarios in a list)
rearranged_result_tree = experiment.result_to_dict(
    include_output=False, alternative_hierarchy=alt_hierarchy
)

print(json.dumps(rearranged_result_tree, indent=2))

In [5]:
# or to a single scenario
rearranged_result_tree = experiment.scenarios[0].result_to_dict(
    alternative_hierarchy=alt_hierarchy
)

rearranged_result_tree

In [8]:
import pandas as pd

experiment.scenarios[0].results_to_csv("s1.csv", level_names=["root", "tech", "activity"])
pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()  # delete the file again

In [9]:
import pandas as pd

experiment.scenarios[0].results_to_csv(
    "s1.csv", alternative_hierarchy=alt_hierarchy, level_names=["root", "activity"]
)
pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()  # delete the file again